<a href="https://colab.research.google.com/github/aksingh570/trojan-detection/blob/main/TROJAN_MALWARE_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle(1).json ~/.kaggle/

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `cp kaggle(1).json ~/.kaggle/'


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("neelabhkshetry/trojan-classification")


100%|██████████| 26.7M/26.7M [00:00<00:00, 73.8MB/s]

Extracting files...


In [ ]:
!kaggle datasets download -d neelabhkshetry/trojan-classification

Dataset URL: https://www.kaggle.com/datasets/neelabhkshetry/trojan-classification
License(s): unknown
trojan-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip trojan-classification.zip -d trojan-classification

Archive:  trojan-classification.zip
  inflating: trojan-classification/test_data.csv  
  inflating: trojan-classification/train_data.csv  


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [ ]:
data = pd.read_csv('trojan-classification/train_data.csv')
data.head()bios

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Class
0,98470,10.42.0.1-10.42.0.42-53-57265-17,10.42.0.42,57265,10.42.0.1,53,17,02/07/2017 08:20:44,545837,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,57471,10.42.0.1-10.42.0.42-53-17935-17,10.42.0.42,17935,10.42.0.1,53,17,14/07/2017 12:22:19,631665,1,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trojan
2,78607,10.42.0.42-106.39.162.36-32973-80-6,10.42.0.42,32973,106.39.162.36,80,6,17/07/2017 02:27:17,9950468,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trojan
3,248,10.42.0.211-103.235.46.211-33302-443-6,10.42.0.211,33302,103.235.46.211,443,6,11/07/2017 09:14:59,22444897,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trojan
4,79487,10.42.0.42-77.234.44.43-55580-80-6,10.42.0.42,55580,77.234.44.43,80,6,17/07/2017 02:48:17,42536386,2,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trojan


Data Preprocessing and Feature Engineering

In [ ]:
# Assuming 'data' is your DataFrame loaded from the dataset
data['label'] = data['Class'].map({'Benign': 0, 'Trojan': 1})

# Check the actual column names in your DataFrame
print(data.columns)

# Replace the column names in the drop() method with actual names, if needed
# For example, if your Source IP column is named 'Src IP'
# and destination IP is named 'Dst IP', etc:
columns_to_drop = ['Class', 'label', 'Unnamed: 0', 'Flow ID',
                   'Src IP', 'Src Port', 'Dst IP', 'Dst Port',  # Replace with actual names
                   'Protocol', 'Timestamp']  # Replace with actual names

X = data.drop(columns=columns_to_drop, errors='ignore')

y = data['label']

# Separate static and dynamic features based on column naming convention (replace with actual names if needed)
# **Modified:** Exclude columns containing strings like IP addresses from scaling
static_features = X.select_dtypes(include=np.number).filter(regex='^Flow Duration|^Total Fwd Packets|...')  # Add all static feature patterns here, selecting only numeric columns
dynamic_features = X.select_dtypes(include=np.number).filter(regex='^Active|^Idle')  # Add all dynamic feature patterns here, selecting only numeric columns


# Standardize the feature values
scaler = StandardScaler()
static_features = scaler.fit_transform(static_features)
dynamic_features = scaler.fit_transform(dynamic_features)

# Split data into training and testing sets
X_train_static, X_test_static, y_train, y_test = train_test_split(static_features, y, test_size=0.3, random_state=42)
X_train_dynamic, X_test_dynamic, _, _ = train_test_split(dynamic_features, y, test_size=0.3, random_state=42)

Index(['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
       ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Len

Layered classification

In [ ]:
# Static classifier - Random Forest
static_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
static_classifier.fit(X_train_static, y_train)

# Static Layer Prediction
static_predictions = static_classifier.predict(X_test_static)
print("Static Layer Classification Report:\n", classification_report(y_test, static_predictions))


Static Layer Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.75      0.73     23404
           1       0.74      0.71      0.72     24516

    accuracy                           0.73     47920
   macro avg       0.73      0.73      0.73     47920
weighted avg       0.73      0.73      0.73     47920



In [ ]:
# Dynamic classifier - Multi-Layer Perceptron (MLP)
dynamic_classifier = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=100, random_state=42)
dynamic_classifier.fit(X_train_dynamic, y_train)

# Dynamic Layer Prediction
dynamic_predictions = dynamic_classifier.predict(X_test_dynamic)
print("Dynamic Layer Classification Report:\n", classification_report(y_test, dynamic_predictions))

Dynamic Layer Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.91      0.65     23404
           1       0.61      0.13      0.22     24516

    accuracy                           0.51     47920
   macro avg       0.55      0.52      0.43     47920
weighted avg       0.55      0.51      0.43     47920



hybrid Analysis

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

# ... (your existing code to define and fit static_classifier and dynamic_classifier) ...

# Combining static and dynamic classifiers
hybrid_classifier = VotingClassifier(
    estimators=[('static', static_classifier), ('dynamic', dynamic_classifier)],
    voting='soft'  # Use soft voting to take probability scores into account
)

# Train on a combined dataset (here, using concatenated features)
X_train_combined = np.hstack((X_train_static, X_train_dynamic))
X_test_combined = np.hstack((X_test_static, X_test_dynamic))

# Impute NaN values using SimpleImputer
imputer = SimpleImputer(strategy='mean') # or 'median', 'most_frequent', etc.
X_train_combined = imputer.fit_transform(X_train_combined)
X_test_combined = imputer.transform(X_test_combined)

hybrid_classifier.fit(X_train_combined, y_train)
hybrid_predictions = hybrid_classifier.predict(X_test_combined)
print("Hybrid Layer Classification Report:\n", classification_report(y_test, hybrid_predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Hybrid Layer Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.75      0.72     23404
           1       0.74      0.69      0.71     24516

    accuracy                           0.72     47920
   macro avg       0.72      0.72      0.72     47920
weighted avg       0.72      0.72      0.72     47920



In [ ]:
# Individual layer evaluation
print("Static Layer Accuracy:", accuracy_score(y_test, static_predictions))
print("Dynamic Layer Accuracy:", accuracy_score(y_test, dynamic_predictions))

# Final hybrid model evaluation
print("Hybrid Model Accuracy:", accuracy_score(y_test, hybrid_predictions))
print("Full Classification Report:\n", classification_report(y_test, hybrid_predictions))


Static Layer Accuracy: 0.7257095158597663
Dynamic Layer Accuracy: 0.5117487479131887
Hybrid Model Accuracy: 0.717737896494157
Full Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.75      0.72     23404
           1       0.74      0.69      0.71     24516

    accuracy                           0.72     47920
   macro avg       0.72      0.72      0.72     47920
weighted avg       0.72      0.72      0.72     47920

